Data INTRO Source of Air Pollution and its effect on Respiratory Health:<br>
https://www.epa.gov/pmcourse/particle-pollution-and-respiratory-effects

- Limit the size of queries. Our database contains billions of values and you may request more than you intend. If you are unsure of the amount of data, start small and work your way up. We request that you limit queries to 1,000,000 rows of data each. **You can use the "observation count" field on the annualData service to determine how much data exists for a time-parameter-geography combination.** If you have any questions or need advice, please contact us.
- Limit the frequency of queries. Our system can process a limited load. If scripting requests, please wait for one request to complete before submitting another and do not make more than 10 requests per minute. Also, we request a pause of 5 seconds between requests and adjust accordingly for response time and size.

In [14]:
import requests
import json
import pandas as pd
import re
import time

In [15]:
with open('../data/credentials.json') as file:
    credentials = json.load(file)
epa_key = credentials['epa_key']
epa_email = credentials['epa_email']

#### Declare the Static Variables

In [16]:
# State Level FIPS Codes
fips_ca = '06'
fips_co = '08'
fips_ga = '13'
fips_tn = '47'

# County Level FIPS Codes
fips_sd = '073'
fips_la = '037'
fips_den = '031'
fips_ful = '121'
fips_dek = '089'
fips_dav = '037'

fips_dict = {fips_ca: [fips_sd, fips_la],
             fips_co: [fips_den],
             fips_ga: [fips_ful, fips_dek],
             fips_tn: [fips_dav]}

# Request Dates
date_list = ['20110101', '20160101', '20210101']

#### Obtain the AQS (Air Quality System) Parameter Codes
AQI = Air Quality Index<br>
- The AQI is a nationally uniform color-coded index for reporting and forecasting daily air quality. It is used to report on the most common ambient air pollutants that are regulated under the Clean Air Act: **ground-level _ozone_, particle pollution (_PM10 and PM2.5_), _carbon monoxide (CO)_, _nitrogen dioxide (NO2)_, and _sulfur dioxide (SO2)_.** The AQI tells the public how clean or polluted the air is and how to avoid health effects associated with poor air quality.
- The AQI focuses on health effects that may be experienced within a few hours or days after breathing polluted air and uses a normalized scale from 0 to 500; the higher the AQI value, the greater the level of pollution and the greater the health concern. An AQI value of 100 generally corresponds to the level of the short-term National Ambient Air Quality Standard for the pollutant. **AQI values at and below 100 are generally considered to be satisfactory. When AQI values are above 100, air quality is considered to be unhealthy, at first for members of populations at greatest risk of a health effect, then for the entire population as AQI values get higher (greater than 150).**

#### Filter the Parameter Codes

In [24]:
param_codes = pd.read_csv('../data/parameters.csv')
param_codes = param_codes.rename(columns={'Parameter Code': 'code', 
                                          'Parameter': 'param',
                                          'Parameter Abbreviation': 'abbr',
                                          'Still Valid': 'valid',
                                          'Standard Units':'std_units'})
param_codes = param_codes[['code', 'param', 'abbr', 'valid', 'std_units']].sort_values(by='code').reset_index(drop=True)
param_codes['code'] = param_codes.code.astype('string')

In [7]:
param_codes.loc[param_codes.abbr.str.contains('^[CSN]{1}O2{0,1}$|SMKE|^O3$|^BZ$', regex=True, na=False, case=False)]

,code,param,abbr,valid,std_units
13,11204,Smoke,SMKE,YES,Micrograms/cubic meter (25 C)
534,42101,Carbon monoxide,CO,YES,Parts per million
535,42102,Carbon dioxide,CO2,YES,Parts per million
549,42401,Sulfur dioxide,SO2,YES,Parts per billion
555,42601,Nitric oxide (NO),NO,YES,Parts per billion
556,42602,Nitrogen dioxide (NO2),NO2,YES,Parts per billion
855,44201,Ozone,O3,YES,Parts per million
866,45201,Benzene,BZ,YES,Parts per billion Carbon


In [9]:
param_codes.loc[param_codes.param.str.contains('^PM2[\.]*5|^LC10|^PM10', regex=True, na=False, case=False)]

,code,param,abbr,valid,std_units
1029,81102,PM10 Total 0-10um STP,PM10,YES,Micrograms/cubic meter (25 C)
1031,81104,PM2.5 STP,PM2.5,YES,Micrograms/cubic meter (25 C)
1173,85101,PM10 - LC,LC10,YES,Micrograms/cubic meter (LC)
1306,88101,PM2.5 - Local Conditions,LC25,YES,Micrograms/cubic meter (LC)
1449,88502,Acceptable PM2.5 AQI & Speciation Mass,PM2.5,YES,Micrograms/cubic meter (LC)


In [25]:
param_codes.loc[~param_codes.abbr.isnull()]

,code,param,abbr,valid,std_units
0,11101,Suspended particulate (TSP),PT,YES,Micrograms/cubic meter (25 C)
2,11103,Benzene soluble organics (TSP),BZSOP,YES,Micrograms/cubic meter (25 C)
3,11104,Total polynuclear hydrocarbons,PNHC,YES,Micrograms/cubic meter (25 C)
4,11114,Windblown particulate,WBPT,YES,Particles/sq millimeter/week
10,11201,Soil index (COH),COH,YES,"COHS/1,000 linear feet"
...,...,...,...,...,...
1435,88388,OP CSN_Rev Unadjusted PM2.5 LC TOT,OPTT_r,YES,Micrograms/cubic meter (LC)
1445,88401,Reconstructed Mass PM2.5 LC,RCMN,YES,Micrograms/cubic meter (LC)
1446,88403,Sulfate PM2.5 LC,SO4,YES,Micrograms/cubic meter (LC)
1449,88502,Acceptable PM2.5 AQI & Speciation Mass,PM2.5,YES,Micrograms/cubic meter (LC)


In [26]:
param_codes.loc[(param_codes.abbr.str.contains('^[CSN]{1}O2{0,1}$|SMKE|^O3$|^BZ$', regex=True, na=False, case=False) |
                              param_codes.param.str.contains('^PM2[\.]*5|^PM10', regex=True, na=False, case=False)) &
                ~param_codes.abbr.isnull()]


,code,param,abbr,valid,std_units
13,11204,Smoke,SMKE,YES,Micrograms/cubic meter (25 C)
534,42101,Carbon monoxide,CO,YES,Parts per million
535,42102,Carbon dioxide,CO2,YES,Parts per million
549,42401,Sulfur dioxide,SO2,YES,Parts per billion
555,42601,Nitric oxide (NO),NO,YES,Parts per billion
556,42602,Nitrogen dioxide (NO2),NO2,YES,Parts per billion
855,44201,Ozone,O3,YES,Parts per million
866,45201,Benzene,BZ,YES,Parts per billion Carbon
1029,81102,PM10 Total 0-10um STP,PM10,YES,Micrograms/cubic meter (25 C)
1030,81103,PM10-2.5 STP,PMC,YES,Micrograms/cubic meter (25 C)


In [27]:
param_codes = param_codes.loc[param_codes['valid'] == 'YES']
param_codes = param_codes.loc[(param_codes.abbr.str.contains('^[CSN]{1}O2{0,1}$|SMKE|^O3$|^BZ$', regex=True, na=False, case=False) |
                               param_codes.param.str.contains('^PM2[\.]*5|^PM10', regex=True, na=False, case=False)) &
                               ~param_codes.abbr.isnull()]

param_codes.sort_values(by='code', inplace=True)
print('Number of parameters:', len(param_codes))
param_codes

Number of parameters: 14


,code,param,abbr,valid,std_units
13,11204,Smoke,SMKE,YES,Micrograms/cubic meter (25 C)
534,42101,Carbon monoxide,CO,YES,Parts per million
535,42102,Carbon dioxide,CO2,YES,Parts per million
549,42401,Sulfur dioxide,SO2,YES,Parts per billion
555,42601,Nitric oxide (NO),NO,YES,Parts per billion
556,42602,Nitrogen dioxide (NO2),NO2,YES,Parts per billion
855,44201,Ozone,O3,YES,Parts per million
866,45201,Benzene,BZ,YES,Parts per billion Carbon
1029,81102,PM10 Total 0-10um STP,PM10,YES,Micrograms/cubic meter (25 C)
1030,81103,PM10-2.5 STP,PMC,YES,Micrograms/cubic meter (25 C)


#### Obtain the Air Quality Data (Limit of 5 Parameter Codes per Request)

In [ ]:
resp_df = pd.DataFrame()
endpoint = 'https://aqs.epa.gov/data/api/annualData/byCounty?'

for date in date_list:
    for state in fips_dict:
        for county in fips_dict[state]:
            
            param_list_full = [c for c in param_codes.code]
            param_list_limit = []

            while len(param_list_full):
                if len(param_list_full) >= 5:
                    param_list_limit = param_list_full[:5]
                    param_list_full = param_list_full[5:]
                else:
                    param_list_limit = param_list_full
                    param_list_full = param_list_full[len(param_list_full):]

                request_params = ','.join(param_list_limit)
                params = {'email': epa_email,
                          'key': epa_key,
                          'param': request_params,
                          'bdate': date,
                          'edate': date,
                          'state': state,
                          'county': county}

                # PERFORM THE REQUEST, WAIT AT LEAST 5 SECONDS BETWEEN REQUESTS
                time.sleep(7)
                resp = requests.get(endpoint, params=params).json()
                if (resp['Header'][0]['status'] == 'Success') & ('Data' in resp.keys()):
                    resp_df = pd.concat([resp_df, pd.DataFrame(resp['Data'])], ignore_index=True)
                else:
                    print('Issue with data retrieval; Reason:', resp['Header'][0]['status'],
                          '\nDate:', date, '-- State:', state, '-- County:', county, '-- Params:', request_params, '\n')

resp_df = resp_df.sort_values(by=['parameter_code', 'state_code', 'county_code']).reset_index(drop=True)    

#### Export the Data as a CSV

In [ ]:
resp_df.to_csv('../data/epa_aqs_data.csv')